In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime
import scipy.optimize as opt

from pandas_datareader import data, wb
from __future__ import division

import arch

plt.style.use('ggplot')
%matplotlib inline

Download daily prices of the S&P500 index, GE and IBM from finance.yahoo.com starting in 1/1/1970.

In [2]:
px_df = pd.read_csv('./yahoo_data.csv')
px_df.head()

,Date,GE,IBM,^GSPC
0,1970-01-02,0.176514,5.444441,93.000000
1,1970-01-05,0.175650,5.496684,93.459999
2,1970-01-06,0.170467,5.500415,92.820000
3,1970-01-07,0.171330,5.504147,92.629997
4,1970-01-08,0.172770,5.515342,92.680000


In [3]:
class yahoo_finance:
    
    def __init__(self, raw_data, t_idx):
        
        self.px = pd.Series(raw_data.values, index=pd.DatetimeIndex(t_idx))
        self.rets = np.log(self.px).diff().dropna() * 100
        self.m_vol = np.sqrt(self.rets.apply(lambda r: r*r).groupby(pd.TimeGrouper(freq='MS')).sum())
        self.m_ret = self.rets.groupby(pd.TimeGrouper(freq='MS')).sum()
    
    def display(self):
        
        return pd.concat([self.px, self.rets], keys=['Adj Close', 'Return'], axis=1)

In [4]:
GE_df = yahoo_finance(px_df['GE'], px_df['Date'])
IBM_df = yahoo_finance(px_df['IBM'], px_df['Date'])
SPX_df = yahoo_finance(px_df['^GSPC'], px_df['Date'])

In [5]:
def LogLikelihood(r, param, GJR=True):
    
    # get parameters and set legal range
    mu = param[0]
    
    if GJR:
        w, alpha, beta, phi = (p for p in param[1:])
    else:
        w, alpha, beta = (p for p in param[1:])
    
    # initialize
    T = r.shape[0]
    eps = r - mu
    
    v = np.zeros(T + 1)
    v[0] = np.mean(r * r)
    
    if GJR:
        for i in range(T):
            v[i + 1] = w + beta * v[i] + (alpha + phi if eps[i] < 0 else alpha) * eps[i] * eps[i]
    else:
        for i in range(T):
            v[i + 1] = w + beta * v[i] + alpha * eps[i] * eps[i]

    e = eps / np.sqrt(v[:-1]) 
    sse = e.T.dot(e)
    
    return -0.5 * float(T * np.log(2 * np.pi) + np.sum(np.log(v[:-1])) + sse)

In [6]:
def output_res(p, GJR=True):
    
    print 'MEAN MODEL'
    print '\tmu = %.4f' %(p[0])
    print 'VOL MODEL'
    if GJR:
        print '\tomega = %.4f \n \talpha = %.4f \n \tbeta = %.4f \n \tphi = %.4f' %(p[1], p[2], p[3], p[4])
    else:
        print '\tomega = %.4f \n \talpha = %.4f \n \tbeta = %.4f' %(p[1], p[2], p[3])

# GARCH(1,1) - constant mean mode

In [7]:
param = np.random.uniform(0, 1, 4)
GARCH_min_object = lambda p: - LogLikelihood(SPX_df.m_ret, p, False)

G11_res = opt.minimize(GARCH_min_object, param, method='nelder-mead',options={'xtol': 1e-8, 'disp': True})
output_res(G11_res.x, False)

Optimization terminated successfully.
         Current function value: 1627.249798
         Iterations: 231
         Function evaluations: 424
MEAN MODEL
	mu = 0.6434
VOL MODEL
	omega = 0.7926 
 	alpha = 0.1230 
 	beta = 0.8431


In [8]:
y = SPX_df.m_ret
mod_G11 = arch.arch_model(y, vol='GARCH', p=1,q=1)
pkg_G11_res = mod_G11.fit()
print(pkg_G11_res.summary())

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1628.52151551
Iteration:      2,   Func. Count:     16,   Neg. LLF: 1627.31390509
Iteration:      3,   Func. Count:     24,   Neg. LLF: 1626.97718964
Iteration:      4,   Func. Count:     32,   Neg. LLF: 1626.90395866
Iteration:      5,   Func. Count:     40,   Neg. LLF: 1626.90119341
Iteration:      6,   Func. Count:     47,   Neg. LLF: 1626.83029101
Iteration:      7,   Func. Count:     53,   Neg. LLF: 1626.8208196
Iteration:      8,   Func. Count:     59,   Neg. LLF: 1626.82056116
Iteration:      9,   Func. Count:     65,   Neg. LLF: 1626.82051692
Iteration:     10,   Func. Count:     71,   Neg. LLF: 1626.82051379
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1626.82051379
            Iterations: 10
            Function evaluations: 71
            Gradient evaluations: 10
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                   Non

# GJR-GARCH(1,1) - constant mean mode

In [9]:
param = np.random.uniform(0, 1, 5)
GJR_min_object = lambda p: - LogLikelihood(SPX_df.m_ret, p)

GJR11_res = opt.minimize(GJR_min_object, param, method='nelder-mead',options={'xtol': 1e-8, 'disp': True})
output_res(GJR11_res.x)

/Users/jenchieh/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: RuntimeWarning: invalid value encountered in sqrt
/Users/jenchieh/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:28: RuntimeWarning: invalid value encountered in log


Optimization terminated successfully.
         Current function value: 1624.782006
         Iterations: 462
         Function evaluations: 768
MEAN MODEL
	mu = 0.5564
VOL MODEL
	omega = 1.2407 
 	alpha = 0.0546 
 	beta = 0.8261 
 	phi = 0.1099


In [10]:
# GJR-GARCH

y = SPX_df.m_ret
mod_GJR11 = arch.arch_model(y = y, vol = 'GARCH', p = 1, o = 1, q = 1)

pkg_GJR11_res = mod_GJR11.fit()
print(pkg_GJR11_res.summary())

Iteration:      1,   Func. Count:      7,   Neg. LLF: 1625.03571925
Iteration:      2,   Func. Count:     16,   Neg. LLF: 1624.84040732
Iteration:      3,   Func. Count:     25,   Neg. LLF: 1624.71403758
Iteration:      4,   Func. Count:     34,   Neg. LLF: 1624.68131665
Iteration:      5,   Func. Count:     42,   Neg. LLF: 1624.58252667
Iteration:      6,   Func. Count:     50,   Neg. LLF: 1624.5297289
Iteration:      7,   Func. Count:     58,   Neg. LLF: 1624.47678149
Iteration:      8,   Func. Count:     66,   Neg. LLF: 1624.43375528
Iteration:      9,   Func. Count:     75,   Neg. LLF: 1624.42495536
Iteration:     10,   Func. Count:     83,   Neg. LLF: 1624.4040733
Iteration:     11,   Func. Count:     90,   Neg. LLF: 1624.40175921
Iteration:     12,   Func. Count:     97,   Neg. LLF: 1624.40100511
Iteration:     13,   Func. Count:    104,   Neg. LLF: 1624.40097751
Iteration:     14,   Func. Count:    111,   Neg. LLF: 1624.40094367
Iteration:     15,   Func. Count:    118,   Neg. L